# RT-IoT2022 - Projeto de Machine Learning
**Objetivo Acadêmico:** Este projeto visa detectar tráfego malicioso em dispositivos IoT, utilizando modelos clássicos de Machine Learning.
A tarefa consiste em classificar o tráfego como benigno ou malicioso, sendo uma aplicação prática de detecção de intrusão em redes IoT.

## 1. Importação de bibliotecas
Nesta etapa, importamos bibliotecas essenciais para manipulação de dados, pré-processamento, modelagem e avaliação de modelos de Machine Learning.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## 2. Carregamento do Dataset
O dataset RT-IoT2022 foi obtido do UCI Machine Learning Repository. Contém aproximadamente 123.000 instâncias e 83 atributos, descrevendo o tráfego de rede de dispositivos IoT, incluindo cenários de ataques e tráfego benigno.

In [ ]:
df = pd.read_csv('RT-IoT2022.csv')
# Visualização inicial
df.head()
df.info()
df.isnull().sum()

## 3. Pré-processamento
Nesta etapa, realizamos a separação entre atributos e rótulo, codificação das classes, e divisão entre conjunto de treino e teste mantendo a proporção das classes originais.

In [ ]:
X = df.drop('label', axis=1)
y = df['label']
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## 4. Criação de Pipelines e Hiperparâmetros
Foram selecionados cinco modelos clássicos de Machine Learning: Random Forest, Gradient Boosting, Logistic Regression, SVM e KNN. Cada modelo será encapsulado em um pipeline que inclui padronização dos atributos e otimização de hiperparâmetros via GridSearchCV.

In [ ]:
pipelines = {
    'rf': Pipeline([('scaler', StandardScaler()), ('clf', RandomForestClassifier(random_state=42))]),
    'gb': Pipeline([('scaler', StandardScaler()), ('clf', GradientBoostingClassifier(random_state=42))]),
    'lr': Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression(max_iter=1000, random_state=42))]),
    'svm': Pipeline([('scaler', StandardScaler()), ('clf', SVC(random_state=42))]),
    'knn': Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())])
}
params = {
    'rf': {'clf__n_estimators':[100,200], 'clf__max_depth':[None,10,20]},
    'gb': {'clf__n_estimators':[100,200], 'clf__learning_rate':[0.01,0.1]},
    'lr': {'clf__C':[0.1,1,10]},
    'svm': {'clf__C':[0.1,1,10], 'clf__kernel':['linear','rbf']},
    'knn': {'clf__n_neighbors':[3,5,7]}
}

## 5. Treinamento e Validação com Cross-Validation
Aplicamos validação cruzada estratificada (5 folds) para garantir avaliação robusta e generalizável dos modelos. GridSearchCV é usado para encontrar os melhores hiperparâmetros.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_models = {}
for name in pipelines:
    grid = GridSearchCV(pipelines[name], params[name], cv=cv, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid.best_estimator_
    print(f'{name} melhor score CV: {grid.best_score_:.4f}')
    print(f'Melhores parâmetros: {grid.best_params_}
')

## 6. Avaliação no Conjunto de Teste
Avaliação dos modelos treinados no conjunto de teste com métricas como acurácia, precision, recall, F1-score e matriz de confusão. Esta etapa permite comparar o desempenho e robustez dos modelos.

In [ ]:
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f'Modelo: {name}')
    print(f'Acurácia: {accuracy_score(y_test, y_pred):.4f}')
    print('Classification Report:')
    print(classification_report(y_test, y_pred))
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    print('-'*50)

## 7. Conclusão
Os resultados obtidos permitem comparar os modelos de ML quanto à performance e selecionar o mais adequado para detecção de tráfego malicioso em IoT.
O notebook está pronto para ser utilizado em apresentações acadêmicas ou upload direto no GitHub.